<a href="https://colab.research.google.com/github/akash166d/sparkNLP/blob/master/LifeIO_TopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed -q spark-nlp==2.5.4

! pip install --user -U nltk

openjdk version "1.8.0_275"
OpenJDK Runtime Environment (build 1.8.0_275-8u275-b01-0ubuntu1~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.275-b01, mixed mode)
     |████████████████████████████████| 215.7MB 58kB/s 
     |████████████████████████████████| 204kB 46.2MB/s 
     |████████████████████████████████| 133kB 4.2MB/s 
     |████████████████████████████████| 1.4MB 4.3MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434678 sha256=e41b0bc7e5b76595dbed762f4ee6c249f444b2ec39d989c23f78cc25cc8fe94e
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
! pip install demoji
import demoji
demoji.download_codes()

... OK (Got response in 0.36 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [ ]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version; ", spark.version)

Spark NLP version:  2.5.4
Apache Spark version;  2.4.4


In [ ]:
! ln -s "/content/drive/My Drive" "/content/MyDrive"

In [ ]:
#dateparse = lambda x: pd.datetime.strptime(x, '%m/%d/%y')
#df_email_temp = pd.read_csv('/content/drive/My Drive/Capstone_LifeIO/LifeIO_Data/lifeio_all_users_email_activity.csv')#,parse_dates=['delivered_date','click_date','open_date'],date_parser=dateparse)
df_email_temp= spark.read.format("csv").option("header", "true").load("/content/drive/My Drive/Capstone_LifeIO/LifeIO_Data/lifeio_all_users_email_activity.csv")
df_email_temp.filter(df_email_temp.email_subject_line != '')
df_email_temp.show()

+----------------+--------------------+------------------+----------+----------+
|       client_id|  email_subject_line|    delivered_date|click_date| open_date|
+----------------+--------------------+------------------+----------+----------+
|1999120100000018|Avoiding exercise...|        2020-07-02|      null|      null|
|1999120100000018|Committing is har...|        2020-07-09|      null|      null|
|1999120100000018|{{ contact.firstn...|        2020-06-18|      null|      null|
|1999120100000018|    Did you miss it❓|        2020-05-22|      null|      null|
|1999120100000018|"⏰ Did you see yo...| """") }} invite?"|2020-06-10|      null|
|1999120100000018|🏃 Don't take ano...|        2020-06-25|      null|      null|
|1999120100000018|Take a class, {{ ...|        2020-06-11|      null|2020-06-11|
|1999120100000018|😲 Uh-oh. Did you...|        2020-05-27|      null|      null|
|1999120100000018|💙 We thought you...|        2020-05-21|      null|      null|
|1999120100000018|🔎 What do you

In [ ]:
df_email_temp.count()
dict(df_email_temp.dtypes)

{'click_date': 'string',
 'client_id': 'string',
 'delivered_date': 'string',
 'email_subject_line': 'string',
 'open_date': 'string'}

In [ ]:
df_regitered = spark.read.format("csv").option("header", "true").load('/content/drive/My Drive/Capstone_LifeIO/LifeIO_Data/lifeio_registered_users.csv')
df_regitered=df_regitered.withColumnRenamed('client_id', 'client_id_reg')
df_regitered.show()

+-----------------+----------------+
|registration_date|   client_id_reg|
+-----------------+----------------+
|       2020-06-17|2016112200000322|
|       2020-01-14|2019122300000084|
|       2020-01-14|2019121700000635|
|       2020-01-24|1999120100237070|
|       2020-01-24|2017120500000207|
|       2020-01-30|1999120100277459|
|       2020-02-26|1999120100394514|
|       2020-02-26|1999120101350923|
|       2020-06-09|2008041600001195|
|       2020-03-03|1999120101371500|
|       2020-03-03|1999120100281395|
|       2020-03-03|2003121700000093|
|       2020-06-16|2003122200140725|
|       2020-03-05|2005072800000769|
|       2020-03-11|2000081900009294|
|       2020-03-13|2013090400000157|
|       2020-01-17|2019120400000336|
|       2020-03-16|2005060900000479|
|       2020-03-18|1999120100250345|
|       2020-04-09|2019092300000376|
+-----------------+----------------+
only showing top 20 rows



In [ ]:
df_master = df_email_temp.join(df_regitered,df_email_temp.client_id == df_regitered.client_id_reg,'left')
df_master.filter(df_master.client_id_reg.isNotNull()).show(10)

+----------------+--------------------+------------------+----------+----------+-----------------+----------------+
|       client_id|  email_subject_line|    delivered_date|click_date| open_date|registration_date|   client_id_reg|
+----------------+--------------------+------------------+----------+----------+-----------------+----------------+
|1999120100001500|🔵 {{ contact.fir...|        2020-05-15|      null|2020-05-15|       2020-05-09|1999120100001500|
|1999120100001500|{{ contact.firstn...|        2020-05-16|      null|2020-05-16|       2020-05-09|1999120100001500|
|1999120100001500|😄  {{ contact.fi...|        2020-05-09|      null|2020-05-09|       2020-05-09|1999120100001500|
|1999120100001500|"⏰ Did you see yo...| """") }} invite?"|2020-05-08|      null|       2020-05-09|1999120100001500|
|1999120100001500|Take a class, {{ ...|        2020-05-09|2020-05-09|2020-05-09|       2020-05-09|1999120100001500|
|1999120100001500|🎉 There's a new ...|        2020-05-20|      null|2020-0

# Create Target Columns

In [ ]:
from pyspark.sql.functions import udf, explode
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import *
import demoji

@udf(StringType())
def ReplaceEmoji(email_str):
  text = email_str
  if text is None:
    return ''
  else:
    return str((demoji.replace(email_str))) 

@udf(IntegerType())
def BinaryTarget(value):
  if (value != value) | (value is None):
       return 0
  else:
      return 1




df_clicked_master = df_master.select('client_id',ReplaceEmoji('email_subject_line').alias('email_sub'),BinaryTarget('click_date').alias('is_clicked'))
df_clicked_master.show(10)


df_opened_master = df_master.select('client_id',ReplaceEmoji('email_subject_line').alias('email_sub'),BinaryTarget('open_date').alias('is_opened'))
df_opened_master.show(10)



df_registred_master = df_master.select('client_id',ReplaceEmoji('email_subject_line').alias('email_sub'),BinaryTarget('registration_date').alias('is_reg'))
df_registred_master.show(10)

+----------------+--------------------+----------+
|       client_id|           email_sub|is_clicked|
+----------------+--------------------+----------+
|1999120100000018|Avoiding exercise...|         0|
|1999120100000018|Committing is har...|         0|
|1999120100000018|{{ contact.firstn...|         0|
|1999120100000018|     Did you miss it|         0|
|1999120100000018|" Did you see you...|         1|
|1999120100000018| Don't take anoth...|         0|
|1999120100000018|Take a class, {{ ...|         0|
|1999120100000018| Uh-oh. Did you f...|         0|
|1999120100000018| We thought you c...|         0|
|1999120100000018| What do you google?|         0|
+----------------+--------------------+----------+
only showing top 10 rows

+----------------+--------------------+---------+
|       client_id|           email_sub|is_opened|
+----------------+--------------------+---------+
|1999120100000018|Avoiding exercise...|        0|
|1999120100000018|Committing is har...|        0|
|199912010

In [ ]:
print('Percentage of clicked')
print((df_clicked_master.filter(df_clicked_master.is_clicked == 1).count()/1033567)*100)

print('Percentage of opened')
print((df_opened_master.filter(df_opened_master.is_opened == 1).count()/1033567)*100)

print('Percentage of registered')
print((df_registred_master.filter(df_registred_master.is_reg == 1).count()/1033567)*100)


Percentage of clicked
4.756440559731493
Percentage of opened
10.389940855309815
Percentage of registered
6.283482348023882


# Click Engagement

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import sum,avg,max,min,mean,count

df_clicked_master = df_clicked_master.groupBy('email_sub').avg('is_clicked').alias('eng_percentage')

df_clicked_master.show()

+--------------------+--------------------+
|           email_sub|     avg(is_clicked)|
+--------------------+--------------------+
| {{ contact.first...| 0.04142692750287687|
| It's time for yo...|0.005819924683327628|
| It's activation ...| 0.02826086956521739|
| {{ contact.first...|0.004131227217496963|
|Committing is har...|0.001673557786965...|
| {{ contact.first...| 0.02524271844660194|
| {{ contact.first...| 0.02711323763955343|
| Welcome to {{ co...| 0.06666666666666667|
| {{ contact.first...| 0.03951367781155015|
| What's nagging you?|0.005755838741768914|
| You're invited. ...| 0.01009405827024547|
| {{ contact.first...| 0.08171206225680934|
| [New] Boost poin...|0.026845637583892617|
| {{ contact.first...|                0.25|
| {{ contact.first...| 0.05067985166872682|
|Finally, an easie...|0.003323485967503...|
| Tell us what you...|                 0.0|
| {{ contact.first...|                0.04|
|{{ contact.firstn...|0.023505572441742655|
| [New] Pay it dow...| 0.0236486

In [ ]:
df_clicked_master.count()

175

In [ ]:
# df_clicked_master.toPandas().to_csv('/content/drive/My Drive/Capstone_LifeIO/eng_clicked.csv')

In [ ]:
splits = [float("-inf"), 0.01, 0.03, float("inf")]
labels = ["Low", "Med","Hig",]

from pyspark.ml.feature import Bucketizer
from pyspark.sql.functions import array, col, lit

bucketizer = Bucketizer(splits=splits, inputCol="avg(is_clicked)", outputCol="is_clicked")

with_split = bucketizer.transform(df_clicked_master)

In [ ]:
with_split.show()

+--------------------+--------------------+----------+
|           email_sub|     avg(is_clicked)|is_clicked|
+--------------------+--------------------+----------+
| {{ contact.first...| 0.04142692750287687|       2.0|
| It's time for yo...|0.005819924683327628|       0.0|
| It's activation ...| 0.02826086956521739|       1.0|
| {{ contact.first...|0.004131227217496963|       0.0|
|Committing is har...|0.001673557786965...|       0.0|
| {{ contact.first...| 0.02524271844660194|       1.0|
| {{ contact.first...| 0.02711323763955343|       1.0|
| Welcome to {{ co...| 0.06666666666666667|       2.0|
| {{ contact.first...| 0.03951367781155015|       2.0|
| What's nagging you?|0.005755838741768914|       0.0|
| You're invited. ...| 0.01009405827024547|       1.0|
| {{ contact.first...| 0.08171206225680934|       2.0|
| [New] Boost poin...|0.026845637583892617|       1.0|
| {{ contact.first...|                0.25|       2.0|
| {{ contact.first...| 0.05067985166872682|       2.0|
|Finally, 

In [ ]:
df_clicked_master = with_split

# NLP: Text Preprocessing

In [ ]:
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from pyspark.sql import functions as F

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
assembler = DocumentAssembler().setInputCol('email_sub').setOutputCol('document')

sentence_detector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")

tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('token')#.setTargetPattern('/\b(\?You were looking)\b/')#.setExceptionsPath('/content/MyDrive/SparkNLP/entities.txt')

spell_chk = NorvigSweetingModel().pretrained().setInputCols(['token']).setOutputCol('corrected')

lemmatizer = LemmatizerModel().pretrained().setInputCols(['corrected']).setOutputCol('lemma')

normalizer = Normalizer().setInputCols(['lemma']).setOutputCol('normalized').setLowercase(True)#.setCleanupPatterns('{{([^}]+)}}',' ')

stop_wrd = list( stopwords.words('english'))
stop_wrd.extend(['personalizationtokencontacthsemailfirstsenddate','contactfirstname']) 
stop_words_cleaner = StopWordsCleaner().setInputCols(["normalized"]).setOutputCol("cleanTokens").setCaseSensitive(False).setStopWords(stop_wrd)

embed = WordEmbeddingsModel.pretrained().setInputCols(['document',"token"]).setOutputCol("embeddings")
embeddings_finisher = EmbeddingsFinisher().setInputCols(["embeddings"]).setOutputCols(["embeddings"]).setOutputAsVector(True).setCleanAnnotations(False)

finisher = Finisher().setInputCols(['cleanTokens']).setOutputCols(['cleanTokens']).setOutputAsArray(True).setCleanAnnotations(True)

pipeline_embd = Pipeline().setStages([ assembler, tokenizer, spell_chk , lemmatizer, normalizer,stop_words_cleaner,embed,embeddings_finisher,finisher ]) 


spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[OK!]
lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [ ]:
model_trans_clicked = pipeline_embd.fit(df_clicked_master) 
model_trans_clicked = model_trans_clicked.transform(df_clicked_master) 
model_trans_clicked.show(10, truncate = False)
# model_trans_clicked.toPandas().to_csv('model_trans_clicked.csv')

+--------------------------------------------------------------------+---------------------+----------+---------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.ml.feature import CountVectorizer, IDF

count_vectorizer = CountVectorizer(inputCol='cleanTokens', outputCol='tf', minDF=1)
#cvmodel = count_vectorizer.fit(model_trans_clicked)
idf = IDF(inputCol='tf', outputCol='idf', minDocFreq=1)
# finisher_idf = Finisher().setInputCols(['features']).setOutputCols(['tfidf']) finisher is for sparkNLP not spark ML
bow_pipeline_clicked = Pipeline().setStages([count_vectorizer, idf])
bow_pipeline_clicked = bow_pipeline_clicked.fit(model_trans_clicked)

bows_clicked = bow_pipeline_clicked.transform(model_trans_clicked)
bows_clicked.show()

+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|           email_sub|     avg(is_clicked)|is_clicked|         cleanTokens|          embeddings|                  tf|                 idf|
+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
| {{ contact.first...| 0.04142692750287687|       2.0| [see, new, feature]|[[0.0,0.0,0.0,0.0...|(236,[7,17,50],[1...|(236,[7,17,50],[3...|
| It's time for yo...|0.005819924683327628|       0.0|      [time, reward]|[[0.0,0.0,0.0,0.0...|(236,[1,6,53],[1....|(236,[1,6,53],[2....|
| It's activation ...| 0.02826086956521739|       1.0|[dts, activation,...|[[0.0,0.0,0.0,0.0...|(236,[1,25,110],[...|(236,[1,25],[2.68...|
| {{ contact.first...|0.004131227217496963|       0.0|     [quit, jan, th]|[[0.0,0.0,0.0,0.0...|(236,[198,226,230...|(236,[198,226,230...|
|Committing is har...|0.001

In [ ]:
from pyspark.sql.functions import udf, explode
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import *

@udf(ArrayType(IntegerType()))
def TF_indices(tf):
      K =  tf.indices.tolist()
      return K

@udf(ArrayType(DoubleType()))
def TF_value(tf):
   if isinstance(tf, SparseVector):
      K =  tf.values.tolist()
      return K

@udf(ArrayType(IntegerType()))
def IDF_indices(idf):
      K =  idf.indices.tolist()
      return K

@udf(ArrayType(DoubleType()))
def IDF_value(idf):
   if isinstance(idf, SparseVector):
      K =  idf.values.tolist()
      return K

@udf(IntegerType())
def TF_len(tf):
      K =  len(tf.indices.tolist())
      return K

@udf(IntegerType())
def IDF_len(idf):
      K =  len(idf.indices.tolist())
      return K

bows_clicked_Val = bows_clicked.select("avg(is_clicked)",'cleanTokens',"is_clicked",'embeddings', TF_indices("tf").alias("TF_Ind"), TF_value("tf").alias("TF_Val"),IDF_indices("idf").alias("IDF_Ind"), IDF_value("idf").alias("IDF_Val"),TF_len("tf").alias("TF_Len"),IDF_len("idf").alias("IDF_Len") )
bows_clicked_Val.show(truncate = True)

+--------------------+--------------------+----------+--------------------+-------------------+--------------------+-------------------+--------------------+------+-------+
|     avg(is_clicked)|         cleanTokens|is_clicked|          embeddings|             TF_Ind|              TF_Val|            IDF_Ind|             IDF_Val|TF_Len|IDF_Len|
+--------------------+--------------------+----------+--------------------+-------------------+--------------------+-------------------+--------------------+------+-------+
| 0.04142692750287687| [see, new, feature]|       2.0|[[0.0,0.0,0.0,0.0...|        [7, 17, 50]|     [1.0, 1.0, 1.0]|        [7, 17, 50]|[3.09104245335831...|     3|      3|
|0.005819924683327628| [dts, time, reward]|       0.0|[[0.0,0.0,0.0,0.0...|        [1, 6, 110]|     [1.0, 1.0, 1.0]|        [1, 6, 110]|[2.68557734525015...|     3|      3|
| 0.02826086956521739|[itys, activation...|       1.0|[[0.0,0.0,0.0,0.0...|       [1, 25, 110]|     [1.0, 1.0, 1.0]|       [1, 25, 110]

In [ ]:
@udf(ArrayType(DoubleType()))
def TFIDF_Rank(tf_val,idf_val):
    tf_val = [float(i) for i in tf_val]
    idf_val = [float(i) for i in idf_val]
    products = [a * b for a, b in zip(tf_val, idf_val)]
    return products

bows_clicked_Mul = bows_clicked_Val.select("avg(is_clicked)",'cleanTokens',"is_clicked",'embeddings',TFIDF_Rank("TF_Val","IDF_Val").alias("TF_IDF_Rank"),"IDF_Ind")
bows_clicked_Mul.show(truncate = True)

+--------------------+--------------------+----------+--------------------+--------------------+-------------------+
|     avg(is_clicked)|         cleanTokens|is_clicked|          embeddings|         TF_IDF_Rank|            IDF_Ind|
+--------------------+--------------------+----------+--------------------+--------------------+-------------------+
| 0.04142692750287687| [see, new, feature]|       2.0|[[0.0,0.0,0.0,0.0...|[3.09104245335831...|        [7, 17, 50]|
|0.005819924683327628|      [time, reward]|       0.0|[[0.0,0.0,0.0,0.0...|[2.68557734525015...|             [1, 6]|
| 0.02826086956521739|[itys, activation...|       1.0|[[0.0,0.0,0.0,0.0...|[2.68557734525015...|            [1, 25]|
|0.004131227217496963|     [quit, jan, th]|       0.0|[[0.0,0.0,0.0,0.0...|[4.47733681447820...|    [198, 226, 230]|
|0.001673557786965...|[commit, hardso, ...|       0.0|[[0.2051299959421...|[4.07187170637004...|      [46, 65, 148]|
| 0.02524271844660194|   [test, knowledge]|       1.0|[[0.0,0.0,

In [ ]:
@udf(ArrayType(IntegerType()))
# # @udf(StringType())
def Top3_pos(rank,ind):
    rank = [float(i) for i in rank]
    ind = [int(i) for i in ind]
    top_ind = sorted(range(len(rank)), key=lambda i: rank[i])[-3:] # this give indices where we have top 3 values in rank
    # top_wrd = [ind[i] for i in top_ind]
    return top_ind

@udf(ArrayType(IntegerType()))
def Top3_wrd(rank,ind):
    rank = [float(i) for i in rank]
    ind = [int(i) for i in ind]
    top_ind = sorted(range(len(rank)), key=lambda i: rank[i])[-3:] # this give indices where we have top 3 values in rank
    top_wrd = [ind[i] for i in top_ind]
    return top_wrd

bows_clicked_wrdlevel = bows_clicked_Mul.select("avg(is_clicked)",'cleanTokens',"is_clicked",'embeddings', Top3_pos("TF_IDF_Rank","IDF_Ind").alias("top3_pos"),Top3_wrd("TF_IDF_Rank","IDF_Ind").alias("top3_wrd"))
bows_clicked_wrdlevel.show()

+--------------------+--------------------+----------+--------------------+---------+---------------+
|     avg(is_clicked)|         cleanTokens|is_clicked|          embeddings| top3_pos|       top3_wrd|
+--------------------+--------------------+----------+--------------------+---------+---------------+
| 0.04142692750287687| [see, new, feature]|       2.0|[[0.0,0.0,0.0,0.0...|[0, 1, 2]|    [7, 17, 50]|
|0.005819924683327628|[itys, time, reward]|       0.0|[[0.0,0.0,0.0,0.0...|   [0, 1]|         [1, 6]|
| 0.02826086956521739|[dts, activation,...|       1.0|[[0.0,0.0,0.0,0.0...|   [0, 1]|        [1, 25]|
|0.004131227217496963|     [quit, jan, th]|       0.0|[[0.0,0.0,0.0,0.0...|[0, 1, 2]|[198, 226, 230]|
|0.001673557786965...|[commit, hardso, ...|       0.0|[[0.2051299959421...|[1, 0, 2]|  [73, 65, 148]|
| 0.02524271844660194|   [test, knowledge]|       1.0|[[0.0,0.0,0.0,0.0...|   [0, 1]|       [64, 95]|
| 0.02711323763955343|               [see]|       1.0|[[0.0,0.0,0.0,0.0...|      [

In [ ]:
from pyspark.sql import functions as F
from operator import itemgetter

vocab = bow_pipeline_clicked.stages[0].vocabulary
# udf_to_words = udf(get_words, ArrayType(StringType()))
# use @udf if you have used that pattern, throughout your pynb
@udf(ArrayType(StringType()))
def get_words(token_in):
    if len(token_in) != 1:
      return itemgetter(*token_in)(vocab)
    else:
      return [vocab[token_in[0]]]
    # return map(vocab.__getitem__, token_in)
    # return vocab[token_in].tolist()

In [ ]:
from collections import Counter
from pyspark.sql.functions import udf, explode
from pyspark.sql.functions import struct, array
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors, VectorUDT


# @udf(ArrayType(ArrayType(FloatType())))
# @udf(ArrayType(FloatType()))
# @udf(StringType())
# @udf(IntegerType())
@udf(VectorUDT())
def get_embd_top3(indices,embedd):
  
  # merged = list(itertools.chain(*embedd))
  # merged = functools.reduce(operator.iconcat, embd_vec, [])
  # mer_floats = [float(np_float) for np_float in merged]
  # ver = Vectors.dense(mer_floats)
  #create list of list from embedding using positions
  # indices = [int(i) for i in indices]
  # embedd = v[1]
  embd_top3 = []
  
  for i in indices:
    sublist = embedd[i].values.tolist()
    sublist = [float(np_float) for np_float in sublist]
    embd_top3.append(sublist)
  
  #flatten out the list
  # merged = list(itertools.chain.from_iterable(embd_top3))
  merged = [j for sub in embd_top3 for j in sub]
  
  #convert to vector and send
  ver = Vectors.dense(merged)

  # tp = type(merged)
  return ver
  # return str(type(ver))
  # return len(merged) 

bows_clicked_embed = bows_clicked_wrdlevel.select("avg(is_clicked)",'cleanTokens',"is_clicked", get_embd_top3("top3_pos", "embeddings").alias("features"),get_words(F.col('top3_wrd')).alias('word'))
# bows_embed_2.na.drop(subset=["features"])
bows_clicked_embed.show(truncate = True)
# bows_wrdlevel_2.select('embeddings').show(truncate = False)

+--------------------+--------------------+----------+--------------------+--------------------+
|     avg(is_clicked)|         cleanTokens|is_clicked|            features|                word|
+--------------------+--------------------+----------+--------------------+--------------------+
| 0.04142692750287687| [see, new, feature]|       2.0|[0.0,0.0,0.0,0.0,...| [new, see, feature]|
|0.005819924683327628| [dts, time, reward]|       0.0|[0.0,0.0,0.0,0.0,...|[time, reward, itys]|
| 0.02826086956521739|  [activation, time]|       1.0|[0.0,0.0,0.0,0.0,...|[time, activation...|
|0.004131227217496963|     [quit, jan, th]|       0.0|[0.0,0.0,0.0,0.0,...|     [th, jan, quit]|
|0.001673557786965...|[commit, hardso, ...|       0.0|[-0.5426399707794...|[donat, commit, h...|
| 0.02524271844660194|   [test, knowledge]|       1.0|[0.0,0.0,0.0,0.0,...|   [test, knowledge]|
| 0.02711323763955343|               [see]|       1.0|[0.0,0.0,0.0,0.0,...|               [see]|
| 0.06666666666666667|[welcome

In [ ]:
from pyspark.sql import functions as F
import itertools
import functools
from pyspark.ml.linalg import Vectors, VectorUDT
# @udf(VectorUDT())
@udf(IntegerType())
def get_Flatvector_len_2(embd_vec):
  # embd_vec = list(embd_vec)
  # merged = list(itertools.chain(*embd_vec))
  # merged = functools.reduce(operator.iconcat, embd_vec, [])
  # mer_floats = [float(np_float) for np_float in merged]
  vec =Vectors.dense(embd_vec)
  return (len(vec))
  # return str(type(mer_floats[0]))
  # return embd_vec[0].tolist()

dataset_clicked_clsf_300 = bows_clicked_embed.select('word','features',get_Flatvector_len_2(F.col('features')).alias('feat_cnt'),"is_clicked")
dataset_clicked_clsf_300.na.drop(subset=["feat_cnt"])
dataset_clicked_clsf_300.na.drop(subset=["features"])
# dataset_clsf_300.na.drop(subset=["label"])
# dataset_clsf.show(truncate = True)
# dataset_clsf = dataset_clsf.groupBy("features").agg(F.count('features'))
dataset_clicked_clsf_300 = dataset_clicked_clsf_300.filter((dataset_clicked_clsf_300.feat_cnt == 300) | (dataset_clicked_clsf_300.is_clicked != ' ') ).select('word','features','is_clicked')
# dataset_clsf_300 = dataset_clsf_300.filter(dataset_clsf_300.label != ' ').select('features','label')
# dataset_clsf_300 = dataset_clsf_300.filter(F.col('features') != ' ')
# dataset_clsf_300.show(dataset_clsf_300.count() )
# dataset_clsf_300.repartition(1).count()

dataset_clicked_clsf_300.show()
# dataset_clsf_300.groupBy("feat_cnt").agg(F.count('feat_cnt')).show()

+--------------------+--------------------+----------+
|                word|            features|is_clicked|
+--------------------+--------------------+----------+
| [new, see, feature]|[0.0,0.0,0.0,0.0,...|       2.0|
| [dts, time, reward]|[-0.1440100073814...|       0.0|
|  [time, activation]|[0.0,0.0,0.0,0.0,...|       1.0|
|     [th, jan, quit]|[0.0,0.0,0.0,0.0,...|       0.0|
|[donut, commit, h...|[0.20512999594211...|       0.0|
|[tell, invite, yo...|[-0.2973499894142...|       1.0|
|[point, wait, con...|[0.0,0.0,0.0,0.0,...|       2.0|
|[new, beast, iphone]|[0.95124000310897...|       1.0|
|[finally, way, life]|[-0.1076700016856...|       0.0|
|          [money, +]|[0.0,0.0,0.0,0.0,...|       0.0|
|[new, downheres, ...|[0.0,0.0,0.0,0.0,...|       1.0|
|[ready, account, ...|[-0.5718399882316...|       2.0|
|[reward, app, dow...|[0.0,0.0,0.0,0.0,...|       2.0|
| [debt, day, tackle]|[-0.3668900132179...|       0.0|
|[think, tell, ple...|[0.0,0.0,0.0,0.0,...|       2.0|
| [blood, 

In [ ]:
# dataset_clicked_clsf_300.toPandas().to_csv('/content/drive/My Drive/Capstone_LifeIO/featureNlabel.csv')

In [ ]:
# import pandas as pd
# sch = StructType([StructField("word",ArrayType(StringType()) , True),StructField("features",ArrayType(FloatType()) , True),StructField("is_clicked",IntegerType() , True)])
# df = pd.read_csv('/content/drive/My Drive/Capstone_LifeIO/featureNlabel.csv')
# df = df[['word','features','is_clicked']]


In [ ]:
# from ast import literal_eval
# df['word'] = df['word'].map(lambda a: literal_eval(a))
# df['features'] = df['features'].map(lambda a: literal_eval(a))
# df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550005 entries, 0 to 550004
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   word        550005 non-null  object
 1   features    550005 non-null  object
 2   is_clicked  550005 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.6+ MB


In [ ]:
# df.head()

,word,features,is_clicked
0,"[get, exercise, avoiding]","[-0.4466800093650818, -0.2660300135612488, 0.4...",0
1,"[donut, commit, hard]","[0.20512999594211578, -0.22888000309467316, 0....",0
2,"[wait, point, contactredeempoints]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
3,"[another, without, step]","[-0.13669000566005707, 0.16266000270843506, 0....",0
4,"[think, use, could]","[-0.1779100000858307, 0.6267499923706055, 0.47...",0


In [ ]:


# sch = StructType([StructField("word",ArrayType(StringType()) , True),StructField("features",ArrayType(FloatType()) , True),StructField("is_clicked",IntegerType() , True)])
# schema = StructType().add("word",ArrayType(StringType()) , True).add("features",ArrayType(FloatType()),True)
dataset_clicked_clsf_300 = spark.read.format("csv").option("header", "true").load("/content/drive/My Drive/Capstone_LifeIO/featureNlabel.csv")




In [ ]:
dataset_clicked_clsf_300.show(10)

+---+--------------------+--------------------+----------+
|_c0|                word|            features|is_clicked|
+---+--------------------+--------------------+----------+
|  0|['get', 'exercise...|[-0.4466800093650...|         0|
|  1|['donut', 'commit...|[0.20512999594211...|         0|
|  2|['wait', 'point',...|[0.0,0.0,0.0,0.0,...|         0|
|  3|['another', 'with...|[-0.1366900056600...|         0|
|  4|['think', 'use', ...|[-0.1779100000858...|         0|
|  5|['get', 'exercise...|[-0.4466800093650...|         0|
|  6|['donut', 'commit...|[0.20512999594211...|         0|
|  7|['wait', 'point',...|[0.0,0.0,0.0,0.0,...|         0|
|  8|['another', 'with...|[-0.1366900056600...|         0|
|  9|['think', 'use', ...|[-0.1779100000858...|         0|
+---+--------------------+--------------------+----------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import split, regexp_replace
from pyspark.sql import functions as F
df2 = dataset_clicked_clsf_300.withColumn("word", split(regexp_replace(F.col("word"), '[\\[\\]]', ""), ","))
df2 = df2.withColumn("features", split(regexp_replace(F.col("features"), '[\\[\\]]', ""), ","))
df2.show()
df2.printSchema()

+---+--------------------+--------------------+----------+
|_c0|                word|            features|is_clicked|
+---+--------------------+--------------------+----------+
|  0|['get',  'exercis...|[-0.4466800093650...|         0|
|  1|['donut',  'commi...|[0.20512999594211...|         0|
|  2|['wait',  'point'...|[0.0, 0.0, 0.0, 0...|         0|
|  3|['another',  'wit...|[-0.1366900056600...|         0|
|  4|['think',  'use',...|[-0.1779100000858...|         0|
|  5|['get',  'exercis...|[-0.4466800093650...|         0|
|  6|['donut',  'commi...|[0.20512999594211...|         0|
|  7|['wait',  'point'...|[0.0, 0.0, 0.0, 0...|         0|
|  8|['another',  'wit...|[-0.1366900056600...|         0|
|  9|['think',  'use',...|[-0.1779100000858...|         0|
| 10|['get',  'exercis...|[-0.4466800093650...|         0|
| 11|['donut',  'commi...|[0.20512999594211...|         0|
| 12|['wait',  'point'...|[0.0, 0.0, 0.0, 0...|         0|
| 13|['another',  'wit...|[-0.1366900056600...|         

In [ ]:
from pyspark.sql.functions import udf, explode
from pyspark.ml.linalg import SparseVector, DenseVector,VectorUDT
from pyspark.sql.types import *
from ast import literal_eval
from pyspark.ml.linalg import Vectors

# @udf(ArrayType(FloatType()))
@udf(VectorUDT())
def str2Float(val):
      K = [float(i) for i in val]
      Vectors.dense(K) 
      return K

dataset_clicked_clsf_300 = df2.select('word', str2Float('features').alias('features'), 'is_clicked')

In [ ]:
dataset_clicked_clsf_300.printSchema()

root
 |-- word: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- is_clicked: string (nullable = true)



In [ ]:
dataset_clicked_clsf_300.show()

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "is_clicked", outputCol = "label" )
pipeline = Pipeline(stages=[ label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(dataset_clicked_clsf_300)
dataset = pipelineFit.transform(dataset_clicked_clsf_300)
selectedCols = ['word','features','label']
dataset = dataset.select(selectedCols)
dataset.show(5)
#dataset.select('label').distinct().collect()

Py4JJavaError: ignored

# Classification

In [ ]:
(trainingData, testData) = dataset_clicked_clsf_300.select('word','features','label').randomSplit([0.7, 0.3], seed = 100)
# print("Training Dataset Count: " + str(trainingData.count()))
# print("Test Dataset Count: " + str(testData.count()))
# train, test = dataset_clsf_300.randomSplit([0.6, 0.4], seed=100)
trainingData.show(10)

AnalysisException: ignored

In [ ]:
trainingData=trainingData.distinct()

In [ ]:
@udf(IntegerType())
def fet_len(tf):
      K =  len(tf.indices.tolist())
      return K
trainingData.select('word','features',fet_len('features').alias('features_len')).show()

Py4JJavaError: ignored

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

# assembler = VectorAssembler().setInputCols(['features']).setOutputCol('features')

lr = LogisticRegression(featuresCol = 'features', labelCol = 'label',maxIter=20, regParam=0.3, elasticNetParam=0   )
lrModel = lr.fit(trainingData )
predictions = lrModel.transform(testData)
predictions.show()

Py4JJavaError: ignored

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)
# predictions.show()

In [ ]:
top_wrd_df = predictions.filter(predictions.label == predictions.prediction)
top_wrd_df.show()
top_wrd_df.count()

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

window = Window.partitionBy(top_wrd_df['label']).orderBy(top_wrd_df['probability'].desc())

top_wrd_df = top_wrd_df.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 10)
top_wrd_df.count()

In [ ]:
top_wrd_df_lst = top_wrd_df.groupBy('label').agg(F.collect_list("token"))
top_wrd_df_lst.show(truncate = False)